<a href="https://colab.research.google.com/github/musaddiq82/Learning/blob/main/RAG_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 87.7 MB/s eta 0:00:00


In [1]:
# We need all of these to make it work
!pip install langchain
!pip install transformers
!pip install sentence-transformers
!pip install faiss-gpu  # Use faiss-gpu since we have a GPU
!pip install pypdf
!pip install accelerate
!pip install bitsandbytes

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.2 MB/s eta 0:00:00


In [7]:
import torch
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Load the documents
# We'll use a DirectoryLoader to load all files from our 'data' folder
loader = DirectoryLoader('sample_data/', glob="**/*", use_multithreading=True, loader_cls=TextLoader)
documents = loader.load()

# If you have PDFs, PyPDFLoader is great. This is a common setup:
# loader = DirectoryLoader('data/', glob="**/*.pdf", loader_cls=PyPDFLoader)
# documents = loader.load()


# 2. Chunk the documents
# We split the loaded documents into smaller chunks for the model to process.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(documents)} documents and split them into {len(chunks)} chunks.")

Loaded 6 documents and split them into 81316 chunks.


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 3. Set up the Embedding Model
# This model turns your text chunks into vectors (lists of numbers)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'} # Use the GPU
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# 4. Create the Vector Store
# This creates the FAISS index in memory and adds your chunks to it.
# This one line does all the embedding and storing!
db = FAISS.from_documents(chunks, embeddings)

print("Vector store created successfully.")

Vector store created successfully.


In [12]:
db

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline

# 5. Set up 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 6. Load the LLM and Tokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.1" # A great open-source model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # This will automatically use the GPU
)

# 7. Create a transformers pipeline
# This is the standard way to run a model with transformers
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)

# 8. Wrap the pipeline in LangChain
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("LLM loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


LLM loaded successfully.


/tmp/ipython-input-2669123999.py:34: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [14]:
from langchain.chains import RetrievalQA

# 9. Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" means it "stuffs" all retrieved chunks into the prompt
    retriever=db.as_retriever(search_kwargs={"k": 3}), # k=3 means it retrieves the top 3 chunks
    return_source_documents=True # Optionally, return the source documents
)

print("RAG chain created. You can now ask questions!")

RAG chain created. You can now ask questions!


In [15]:
# --- ASK YOUR QUESTIONS HERE ---
question = "What is the main idea of my document?"
result = qa_chain.invoke(question)

print(f"Question: {question}")
print(f"Answer: {result['result']}")
print("\n--- Sources ---")
for doc in result['source_documents']:
    print(doc.metadata)

# --- TRY ANOTHER QUESTION ---
question = "What was mentioned about [some specific topic from your PDF]?"
result = qa_chain.invoke(question)
print(f"\nQuestion: {question}")
print(f"Answer: {result['result']}")

Question: What is the main idea of my document?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

This directory includes a few sample datasets to get you started.

*   `california_housing_data*.csv` is California housing data from the 1990 US
    Census; more information is available at:
    https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub

*   `mnist_*.csv` is a small sample of the
    [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), which is
    described at: http://yann.lecun.com/exdb/mnist/

*   `anscombe.json` contains a copy of
    [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet); it
    was originally described in

    Anscombe, F. J. (1973). 'Graphs in Statistical Analysis'. American
    Statistician. 27 (1): 17-21. JSTOR 2682899.

    and o